(논문번호) | 대학 | 학과 | 전공 | 재직여부 | 성명 | 교번 | 풀네임 | Year | Title |Source | Status | DB

In [4]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
from openpyxl import load_workbook
import math
from tqdm import tqdm,trange

pd.options.display.float_format = '{:.0f}'.format
pd.options.display.max_columns = None

pip_list = pd.read_excel("DB/PIP/PIP_2014_2020.xlsx")
scopus_list = pd.read_excel("DB/SCOPUS/scopus_all.xlsx")
authors = pd.read_excel("저자리스트(AUID 2020.7월 3일(최신)).xlsx")
scopus_list['Title'] = scopus_list['Title'].str.lower()

In [5]:
research_list = df(columns=['대학','학과','전공','재직여부','성명','교번','풀네임','Year','Title','Source','Status','DB',
                            #추가요청
                            "논문구분",
                            "학술지색인",
                            "학진등재여부",
                            "학진등재후보",
                            #추가 필드
                           'PNU-ID',
                           'DT(P)',
                           'DT(S)',
                           'DOI',
                           'EID',
                           '임용일자',
                           '퇴직일자',
                            #추가요청2
                            '공동연구자수',
                            '권',
                            '호',
                            '페이지',
                            'ISSN',
                            '학술단체명',
                            '국내외구분',
                            '학문분야',
                            '연구원구분',
                            '책임구분',
                            '과제번호(차수)',
                            '과제번호',
                            '연구과제명',
                            '수행기관',
                            '지원기관',
                            '지원금액'
                           ])

In [6]:
res_num=0
for pro in trange(0,len(authors)): #저자리스트 쭉 검사, 한명의 저자에 대해 
    # 만약 저자의 교직원번호가 pip에 있다면
    if authors['교직원번호'].loc[pro] in pip_list['교직원번호'].tolist(): 
        gyojeek_num = authors['교직원번호'].loc[pro]
        for row in range(0,len(pip_list)): #pip에서
            if pip_list['교직원번호'].loc[row] == gyojeek_num: # 해당 교직원 번호의 모든 논문을 긁어온다.
                # 교직원의 인덱스번호를 받아온다
                index_num = authors['교직원번호'].tolist().index(gyojeek_num)
                if math.isnan(authors['퇴직일자'].loc[index_num]):
                    is_in = "Y"
                else: 
                    is_in = "N"
                research_list.loc[res_num] = [
                    pip_list['대학'].loc[row], # 대학
                    pip_list['학과'].loc[row], # 학과
                    pip_list['전공'].loc[row], # 전공
                    is_in, # 재직여부
                    pip_list['성명'].loc[row], # 성명
                    pip_list['교직원번호'].loc[row], # 교번
                    authors['풀네임'].loc[index_num], # 풀네임
                    int(pip_list['게재년월'].loc[row][:4]), # year
                    pip_list['제목'].loc[row], # title
                    pip_list['게재지명'].loc[row], # source
                    "P-only", # status
                    pip_list['학술지색인'].loc[row], # db
                    # 추가요청
                    pip_list['논문구분'].loc[row],
                    pip_list['학술지색인'].loc[row],
                    pip_list['학진등재여부'].loc[row],
                    pip_list['학진등재후보'].loc[row],
                    # 추가 필드
                    authors['PNU_ID'].loc[index_num],# pnu-id
                    pip_list['논문종류'].loc[row],# dt(p)
                    np.nan,# dt(s)
                    np.nan,# doi
                    np.nan,# eid
                    authors['임용일자'].loc[index_num],# 임용일자
                    authors['퇴직일자'].loc[index_num],# 퇴직일자
                    # 추가요청2
                    pip_list['공동연구자수'].loc[row],
                    pip_list['권'].loc[row],
                    pip_list['호'].loc[row],
                    pip_list['페이지'].loc[row],
                    pip_list['ISSN'].loc[row],
                    pip_list['학술단체명'].loc[row],
                    pip_list['국내외구분'].loc[row],
                    pip_list['학문분야'].loc[row],
                    pip_list['연구원구분'].loc[row],
                    pip_list['책임구분'].loc[row],
                    pip_list['과제번호(차수)'].loc[row],
                    pip_list['과제번호'].loc[row],
                    pip_list['연구과제명'].loc[row],
                    pip_list['수행기관'].loc[row],
                    pip_list['지원기관'].loc[row],
                    pip_list['지원금액'].loc[row]
                    
                ]
                res_num += 1
                if pip_list['제목'].loc[row].lower() in scopus_list['Title'].tolist(): # 만약 긁어온 논문이 scopus에 있으면 BOTH로 바꾼다.
                    index_sco_list = scopus_list[scopus_list['Title']==pip_list['제목'].loc[row].lower()]
                    index_num = pip_list['교직원번호'].loc[row]
                    if index_num in index_sco_list['교직원번호'].tolist(): #저자교번도 동일하다면
                        index_sc = scopus_list['Title'].tolist().index(pip_list['제목'].loc[row].lower())
                        research_list['Status'].loc[res_num-1] = "Both"
                        research_list['DT(S)'].loc[res_num-1] = scopus_list['논문타입'].loc[index_sc]
                        research_list['DOI'].loc[res_num-1] = scopus_list['prism:doi'].loc[index_sc]
                        research_list['EID'].loc[res_num-1] = scopus_list['EID'].loc[index_sc]
                    
                    
        for srow in range(0,len(scopus_list)):
            if scopus_list['교직원번호'].loc[srow] == gyojeek_num: # scopus의 번호가 해당 교직번호인데,
                tmp = pip_list[pip_list['교직원번호']==gyojeek_num]
                if  not scopus_list['Title'].loc[srow].lower() in list(map(str.lower,tmp['제목'].tolist())):# pip에는 없다면 (2차검증)
                    # scopus_list에만 있는 경우
                    index_str = scopus_list['AU-ID'].loc[srow]
                    index_str = index_str.replace("au-id(","",1)
                    index_str = index_str.replace(")","",1)
                    index_num = authors['AU-ID'].tolist().index(int(index_str))
                    if math.isnan(authors['퇴직일자'].loc[index_num]):
                        is_in = "Y"
                    else: 
                        is_in = "N"
                    research_list.loc[res_num] = [
                        authors['대학'].loc[index_num], # 대학
                        authors['학과'].loc[index_num], # 학과
                        np.nan, # 전공
                        is_in, # 재직여부
                        scopus_list['성명'].loc[srow], # 성명
                        authors['교직원번호'].loc[index_num], # 교번
                        authors['풀네임'].loc[index_num], # 풀네임
                        int(str(scopus_list['year'].loc[srow])), # year
                        scopus_list['Title'].loc[srow], # title
                        scopus_list['Source'].loc[srow], # source
                        "S-only", # status
                        np.nan, # db
                        # 추가요청
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        # 추가필드
                        authors['PNU_ID'].loc[index_num],# pnu-id
                        np.nan,# dt(p)
                        scopus_list['논문타입'].loc[srow],# dt(s)
                        scopus_list['prism:doi'].loc[srow],# doi
                        scopus_list['EID'].loc[srow],# eid
                        authors['임용일자'].loc[index_num],# 임용일자
                        authors['퇴직일자'].loc[index_num],# 퇴직일자
                        # 추가요청2
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan
                    ]
                    res_num += 1
                


  4%|███▏                                                                            | 71/1771 [01:26<34:18,  1.21s/it]


  8%|██████▎                                                                        | 142/1771 [02:55<37:31,  1.38s/it]


 13%|██████████▏                                                                    | 227/1771 [04:52<39:01,  1.52s/it]


 17%|████████████▉                                                                | 297/1771 [07:20<1:00:06,  2.45s/it]


 21%|████████████████                                                             | 370/1771 [09:55<1:00:51,  2.61s/it]


...

 84%|█████████████████████████████████████████████████████████████████▍            | 1485/1771 [51:45<11:38,  2.44s/it]


 88%|████████████████████████████████████████████████████████████████████▎         | 1551/1771 [55:46<05:52,  1.60s/it]


 93%|████████████████████████████████████████████████████████████████████████▎     | 1641/1771 [59:28<13:01,  6.01s/it]


 96%|█████████████████████████████████████████████████████████████████████████▎  | 1707/1771 [1:04:27<06:59,  6.56s/it]


100%|████████████████████████████████████████████████████████████████████████████| 1771/1771 [1:08:37<00:00,  2.32s/it]


In [7]:
research_list.to_excel("작업용엑셀(2020.07.04).xlsx")